In [ ]:
import gdelt
import requests
import pandas as pd
import numpy as np
import json
import plotly.express as px
from tqdm import tqdm
from datetime import datetime, timedelta, date

# view all columns
pd.set_option('display.max_columns', None)

gd1 = gdelt.gdelt(version=1)

### Load results

In [ ]:
res_dict = {}
specific_string = 'ENV_OIL'

yesterday = datetime.today() - timedelta(days=1)

for i in tqdm(range(50)):
    today = yesterday - timedelta(days=i)

    results = gd1.Search(today.strftime("%Y%m%d"), table='gkg')

    results['THEMES'] = results['THEMES'].str.split(';')

    results = results.dropna(subset=['THEMES']).reset_index()

    filtered_df = results[results['THEMES'].apply(lambda x: specific_string in x)].reset_index()

    filtered_df['TONE_VAL'] = filtered_df['TONE'].str.split(',').apply(lambda x: np.mean([float(item) for item in x]))

    value = filtered_df['TONE_VAL'].mean()
    res_dict[today] = value

    print(today, value)
    
print(res_dict)


In [ ]:

# filter out all rows containing a OIL theme
df = pd.DataFrame(res_dict.items(), columns=['Date', 'Tone'])

# save as csv
df.to_csv('oil_tone.csv', index=False)

df.plot(x='Date', y='Tone', kind='line')